In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt
import numpy as np
import glob,os

# font_path = "/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc"
# prop = mfm.FontProperties(fname=font_path)

In [2]:
data_dir ='./results/'

## loading data

In [3]:
# labels
L = np.load(data_dir + "/L.npy")
# data from torch
tor_data = np.load(data_dir+ "/tor_data.npy")
# data after tsne
X_embedded = np.load(data_dir + "/X_embedded.npy")
#square difference from the center
SE = np.load(data_dir+ "/SE.npy")

subgroup_hier= np.load(data_dir+ "/subgroup_hier.npy", allow_pickle=True)

### load k-mean results

In [4]:
U = np.load(data_dir+"k_mean_15classes.npy",allow_pickle=True)
cl = U.item().get("cl")
c =  U.item().get("c")
k =  U.item().get("k")

In [5]:
### print out k-mean results

In [6]:
text = ''
array_text = []
for i in range(k):
#     print("group%d:"%i, np.array(L)[np.where(cl.numpy()==i)])
#     print(str(np.array(L)[np.where(cl.numpy()==i)]))
    text += "group%d:"%i + str(np.array(L)[np.where(cl.numpy()==i)]) +'\r\n'
    array_text.append(np.array(L)[np.where(cl.numpy()==i)])



In [7]:
# fig = plt.figure(figsize=(10,10))
# cmap = plt.cm.get_cmap("Dark2", k)
# ax = fig.add_subplot(111)
# ax.scatter(X_embedded[:,0], X_embedded[:,1], 
#            c = cmap(cl.cpu().numpy()), alpha = 0.5)
# cmap = plt.cm.get_cmap("Dark2", k)
# # for i in range(len(X_embedded[:,0])):
   
# #     if i%5==0:
# #         ax.text(X_embedded[i,0], X_embedded[i,1],
# #                  L[i], color = cmap(cl.cpu()[i].numpy()),
# #                  size = 14, alpha = 0.8)

# plt.xticks([],[])
# plt.yticks([],[])
# plt.show()
# # plt.savefig(data_saving_path + "/cluster_results_Tsn_k_%d.png"%k)

# Putting meaning into the node in graph

In [8]:
import pinyin
import pinyin.cedict
import textwrap
print(pinyin.get(L[0]))
s = str(pinyin.cedict.translate_word(L[0]))
print(textwrap.fill(s,25))

jiǒng
['variant of 冏[jiong3]',
'used as emoticon
("smiley") meaning
embarrassed, sad :-(,
depressed or frustrated']


In [9]:
defs = []
ps = []
nones = 0
filterL = []
for i in range(len(L)):
    
    s = str(pinyin.cedict.translate_word(L[i]))
    definition = textwrap.fill(s,35)   
    
    if definition == 'None':
        nones += 1
    else:
        filterL.append(i)
    
        Pinyin = str(pinyin.get(L[i]))
        defs.append(definition)
        ps.append(Pinyin)


# Plot using Bokeh

In [10]:
from bokeh.transform import linear_cmap
from bokeh.palettes import Category20
from bokeh.io import output_file, show, output_notebook
from bokeh.models import ColumnDataSource, Plot, ColorBar, OpenURL, TapTool, CustomJS,Range1d,LabelSet, Label
from bokeh.plotting import figure, save
from bokeh.models.glyphs import ImageURL
from bokeh.layouts import row

from random import random
from bokeh.models import CustomJS, ColumnDataSource, Row, Column
from bokeh.plotting import figure, show, curdoc,reset_output
from bokeh.models.widgets import Button, TableColumn, Div,DataTable,TextInput


In [11]:
den_add = "/static/dengrograms"
colors = cl.cpu()[filterL].tolist()
files = [den_add +'/group_%d.png'%i for i in colors]
print(files[0])

/static/dengrograms/group_0.png


In [12]:
subgroup_hier_array =[]
for i in range(15):
    subgroup_hier_array.append(np.array(subgroup_hier.item()[i]))
Sgroups = [subgroup_hier_array[colors[i]] for i in range(len(colors))]

In [13]:
print(len(Sgroups))

1515


In [14]:
classes = colors.copy()
# print(max(colors))
print(len(files))

1515


In [15]:
sub_inds = []
for i,word in enumerate(np.array(L)[filterL].tolist()):
#     print(i,word,Sgroups[i])
    for ind,lists in enumerate(Sgroups[i]):
        for j in lists:
            if j==word:      
                
                sub_inds+=[ind]
#                 print(sub_inds)
                break
        
#     sys.exit()
        

In [16]:

sizes = 1.5*(np.max(SE)-np.array(SE))[filterL]
print(sizes)
class_by_eye = np.load('./results/class_by_eye.npy',allow_pickle=True)

[11.34615066 15.55605584 14.39794764 ...  6.82277785 10.6564673
 12.94882979]


In [17]:
x_text = []
y_text = []
l_text = []
c_text = []
for i in range(len(X_embedded[:,0])):
   
    if i%5==0:
        x_text += [X_embedded[i,0]] 
        y_text += [X_embedded[i,1]]
        l_text += [L[i]]
        c_text += [cl.cpu()[i].numpy()]
                 
print(len(x_text))


390


In [18]:
label_source = ColumnDataSource(data = dict(
    x_text = x_text, 
    y_text = y_text,
    l_text = l_text,
    c_text = c_text,

))
labels = LabelSet(x='x_text', y='y_text', text='l_text', level='glyph',
              x_offset=5, text_alpha = 0.5, y_offset=5, source=label_source,\
                  render_mode='canvas',text_font_size='15pt')

In [19]:
print(class_by_eye[0])

['匚', '冂', '囗']


In [20]:
class_component = [','.join(class_by_eye[colors[i]]) for i in range(len(colors))]
print(len(class_component))
print(class_component[0])

1515
匚,冂,囗


In [21]:
output_file('plot1.html')
TOOLTIPS = """     
<div> 
    <div>
        <span style="font-size: 17px; font-weight: bold;">@desc</span>
        <span style="font-size: 15px; color: #966;">[$index]</span>
    </div>
    <div>
        <span style="font-size: 15px;">Pinyin:</span>
        <span style="font-size: 15px; color: #696;">(@pinyin)</span>
    </div>
    <div>
        <span style="font-size: 15px;">Class</span>
        <span style="font-size: 15px; color: #696;">(@colors)</span>

    </div>
    <div>
        <span style="font-size: 15px;">root logogram</span>
        <span style="font-size: 15px; color: #696;">(@common_parts)</span>

    </div>
    <div>
        <span style="font-size: 15px;">Def</span>
        <span style="font-size: 10px;white-space: pre-line">(@meaning)</span>
    </div>     

</div>
"""

st = ColumnDataSource(data = dict(
    desc=[],
    classes=[],
    words=[],
    meaning=[],

    pinyin = [],
))

s1 = ColumnDataSource(data=dict(
    x = X_embedded[:,0][filterL],
    y = X_embedded[:,1][filterL],
    desc=np.array(L)[filterL].tolist(),

    colors = colors,
    meaning = defs,
    pinyin = ps,
    sizes = list(sizes),
    classes = classes,
    common_parts = class_component,
    images_files = files,
    words=Sgroups,
    Sinds = sub_inds,

))

s2 = ColumnDataSource(data=dict(images_files=[]))
s1.selected.js_on_change('indices', CustomJS(args=dict(s1=s1, s2=s2, st=st), code="""
    var inds = cb_obj.indices;
    var d1 = s1.data;
    var d2 = s2.data;
    var d3 = st.data;

    console.log(inds)
    

    d2['images_files'] = []
    d2['images_files'].push(d1['images_files'][inds[0]])
    console.log(d2['images_files'])
    for (var i = 0; i < inds.length; i++) {
        

        d3['pinyin'].push(d1['pinyin'][inds[i]]);
        d3['meaning'].push(d1['meaning'][inds[i]]);
        d3['classes'].push(d1['classes'][inds[i]]);
        d3['desc'].push(d1['desc'][inds[i]]);
        
        
        var s_ind = d1['Sinds'][inds[i]];
        console.log(s_ind)
        
        d3['words'].push(d1['words'][inds[i]][s_ind]);


    }
    s2.change.emit();
    st.change.emit();
""")
)



cb_click = CustomJS(args=dict(s1=s1), code="""
    
    var user_input = cb_obj.value
    var d1 = s1.data
    desc = d1['desc']
    
    console.log(user_input)
    for (var i = 0; i < d1['x'].length; i++) {
        if(desc[i] == user_input) {
            var idx = i;
            
            break;
           } 
        }
    console.log(idx)

    l_selected = s1.selected
    
    console.log(s1.selected['1d'])
    console.log(l_selected.indices)
    s1.selected = l_selected

    console.log(s1.selected.indices)
    s1.selected.indices = [idx]
    s1.data = s1.data
    s1.selected.change.emit()

    """)
mapper = linear_cmap(field_name='colors', palette=Category20[max(colors)] ,low=min(colors) ,high=max(colors))

text_input = TextInput(value="", title="Search character here:",callback = cb_click)

p1 = figure(plot_width=800, plot_height=600, tooltips=TOOLTIPS, tools="tap, pan, wheel_zoom, box_zoom, reset",\
            toolbar_location="above", \
            title="Cluster of Chinese characters")

p1.circle('x', 'y', size = 'sizes', fill_color=mapper, line_color=None, fill_alpha=0.75, source=s1)
color_bar = ColorBar(color_mapper=mapper['transform'],
                     orientation='vertical',
                     width=12,  location=(0,0))

p1.add_layout(color_bar, 'right')
# p1.add_layout(labels)



p2 = figure(plot_width=400, plot_height=400,  x_range=(-50, 50), y_range=(-50, 50), \
            title="viewing words within the same class")

p2.image_url(url="images_files", source = s2, x=-60, y=60, w=120, h=120)


columns = [
    TableColumn(field="desc", title="word",width=5),
    TableColumn(field="pinyin", title="pinyin",width=5),
    TableColumn(field="meaning", title="meaning",width=15),
    TableColumn(field="classes", title="class",width=5),
    TableColumn(field="words", title="subclass it in",width=100),

]

data_table = DataTable(source=st, columns=columns, width=700, height=500,reorderable=True,\
                       editable=True,selectable=True, fit_columns=True, scroll_to_selection= True)

layout = Column(row(p1, Column(text_input,p2)), data_table)
#layout = Column(text_input, row(p1, p2))

show(layout)


In [22]:
print([len(array_text[i]) for i in range(15)])

[89, 106, 143, 110, 225, 161, 160, 79, 115, 142, 64, 190, 39, 83, 244]


In [23]:
reset_output()

In [24]:
mapper

{'field': 'colors', 'transform': LinearColorMapper(id='1010', ...)}